In [ ]:
%matplotlib inline
import numpy as np
from IPython.display import display, Video

from ophys_etl.modules.segmentation.qc_utils.classifier_utils import (
    Classifier_ROISet)

from ophys_etl.modules.segmentation.qc_utils.video_display_generator import (
    VideoDisplayGenerator)

video_gen = VideoDisplayGenerator()

The classifier runs off of an HDF5 file that contains some precomputed segmentation artifacts that are expensive to compute on the fly. The file contains video data, image data, and a set of ROIs produced by the segmenter.

In [ ]:
artifact_path='/Users/scott.daniel/Pika/ophys_etl_2/sfd_sandbox/artifacts/output.h5'

The `log_path` kwarg used when instantiating the classifier is the path to a JSON file where lists of valid and invalid ROIs will be stored, along with the path to the artifact file that drove this labeling job.

By default, the classifier will raise an exception if you try to overwrite an existing log file. If you really want to overrwrite an existing log file, instantiate the classifier with `clobber=True`

In [ ]:
log_path = 'dummy_log.json'

In [ ]:
classifier = Classifier_ROISet(artifact_path, log_path=log_path, clobber=True)

Let's just start by viewing all of the ROIs in the field of view

In [ ]:
display(classifier.get_summary_figure())

By specifing an `origin` and a `frame_shape`, we can zoom in on a particular region in the field of view.

The `label_rois` kwarg overplots the ROI IDs (used later) in the field of view. `fontsize` controls the size of these labels.

**Note:** because of image processing conventions, `origin` and `frame_shape` are of the form `(rows, columns)`, which is the transpose of a more traditional `(x, y)` convention.

In [ ]:
display(classifier.get_summary_figure(origin=(300,200), frame_shape=(90, 90), label_rois=True, fontsize=30))

The classifier's `get_roi` method can select one of these ROIs by ROI ID and return it as an object with dedicated methods for producing trace plots, max projection plots, and thumbnail movies.

As an example, let's get the ROI with ROI ID = 191

In [ ]:
roi = classifier.get_roi(191)

`roi.get_trace_plot` will plot the trace of the ROI (the horizontal axis is unitless timesteps; the vertical axis is the average pixel value in the ROI at any given timestep)

In [ ]:
display(roi.get_trace_plot())

By passing in the `timesteps` kwarg, we can zoom in on a specific region of the trace

In [ ]:
display(roi.get_trace_plot(timesteps=np.arange(5000,7000)))

`roi.get_max_projection` provides max projection images centered on the ROI. The following kwargs allow you to control what you seed.

`padding`: an integer that controls how may pixels to either side are included in the field of view.
`include_boundary`: if `True`, the boundary of *this* ROI will be plotted in red over the max projection image.
`include_others`: if `True`, the boundary of all ROIs in the selected field of view will be plotted as well.

In [ ]:
display(roi.get_max_projection(padding=32, include_boundary=False, include_others=False))

In [ ]:
display(roi.get_max_projection(padding=32, include_boundary=True, include_others=False))

In [ ]:
display(roi.get_max_projection(padding=32, include_boundary=True, include_others=True))

`roi.get_thumbnail_video` will instantiate a thumbnail video that can be viewed in the notebook. It also accepts `include_boundary`, `include_others`, and `padding` kwargs. Additionally, it accepts the `timesteps` kwarg to select a specific window in time.

As in the max projection image, the ROI in question is outlined in red. All other ROIs (if requested) are colored according a scheme that prevents bordering ROIs from sharing the same color.

**Note:** The amount of time it takes to generate a thumbnail video depends on how many pixels are included in the field of view (`padding`) and how many timesteps are included. It is not recommended to generate a thumbnail video without specifying a limited number of `timesteps`.

**Note:** Because of the way Jupyter notebooks work, the thumbnail video is written to a temp directory and then symlinked to the local directory of the notebook. The class that instantiates the thumbnail video is smart enough to delete the `mp4` file created by the thumbnail video generator when it passes out of scope. Because of this, the thumbnail video must be instantiated and then passed to `Video(**videogen.display_video(thumbnail_video))` as below. As soon as `thumbnail_video` goes out of scope, you will not be able to view that video without recreating it.

In [ ]:
%%time
blank_thumbnail = roi.get_thumbnail_video(timesteps=np.arange(5000,7000),
                                          padding=32,
                                          include_boundary=False,
                                          include_others=False)
Video(**video_gen.display_video(blank_thumbnail))

In [ ]:
%%time
just_roi_thumbnail = roi.get_thumbnail_video(timesteps=np.arange(5000,7000),
                                          padding=32,
                                          include_boundary=True,
                                          include_others=False)
Video(**video_gen.display_video(just_roi_thumbnail))

In [ ]:
%%time
all_roi_thumbnail = roi.get_thumbnail_video(timesteps=np.arange(5000,7000),
                                          padding=32,
                                          include_boundary=True,
                                          include_others=True)
Video(**video_gen.display_video(all_roi_thumbnail))

The ROI class provides methods `mark_valid` and `mark_invalid` that will record the ROI as either valid or invalid in the log file you specified with your classifier.

**No effort is made to prevent you from labeling the same ROI more than once.***

In [ ]:
roi.mark_valid()

As a demonstration, let's label a few more ROIs.

In [ ]:
roi = classifier.get_roi(672)
roi.mark_invalid()

roi = classifier.get_roi(616)
roi.mark_invalid()

roi = classifier.get_roi(71)
roi.mark_valid()

We can see that this worked by using Pythons `json` module to read in our dummy log and see that these ROIs have been recorded accordingly.

In [ ]:
import json

In [ ]:
with open(log_path, 'rb') as in_file:
    results = json.load(in_file)

In [ ]:
for roi in results['valid_rois']:
    print(roi['id'], roi['valid'])

In [ ]:
for roi in results['invalid_rois']:
    print(roi['id'], roi['valid'])